<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/JE_sentence_trasnformers_TripletsDataset_Scientific_publications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Declare whether you are on Colab or local
colab = True

In [37]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #install required packages
  #!pip install duckdb -q
  !pip install -U sentence-transformers -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, TripletEvaluator, LabelAccuracyEvaluator

In [39]:
#set path to data in Google Drive
data_path_models = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_team1/SynDet/Triplets_Dataset/eval/"
data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/SynDet/Triplets_Dataset/"

In [40]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

In [41]:
model = SentenceTransformer('sentence-transformers/allenai-specter')

In [42]:
#Load train dataset
train = pd.read_csv(data_path_group+"SynDet_ontology_train_triplets.csv")

In [43]:
train.head()

,Unnamed: 0,entity_id,positive_example,negative_example
0,0,electronic health record,online medical records,postal banking
1,1,archival research,government data sources analysis,action design research
2,2,communication service infrastructure,digital enabled communication computers,model driven development
3,3,survey,quantitative based supervised polling techniques,hierarchical Markov model
4,4,three validities,test convergent and design validity,binary classification


In [44]:
#Load validation dataset
eval = pd.read_csv(data_path_group+"SynDet_ontology_val_triplets.csv")

In [45]:
eval.head()

,Unnamed: 0,entity_id,positive_example,negative_example
0,0,autoregressive moving average model,ARMA modelings,survival analysis
1,1,bibliometric analysis,metascience,replication study
2,2,communication service infrastructure,information and communications technology medi...,IT productivity paradox
3,3,individual participant,IS and information senior users,organizational participant
4,5,three validities,diagnostic diagnostic and incremental validities,multitrait multimethod analyis


In [46]:
#Load validation dataset
test = pd.read_csv(data_path_group+"SynDet_ontology_test_triplets.csv")

In [47]:
train_samples = []
for index, row in train.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'],row['negative_example']])
  train_samples.append(inp_example)

In [48]:
evaluation_samples = []
for index, row in eval.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'], row['negative_example']])                                   
  evaluation_samples.append(inp_example)

In [49]:
test_samples = []
for index, row in test.iterrows():
  inp_example = InputExample(texts=[row['entity_id'], row['positive_example'], row['negative_example']])                                   
  test_samples.append(inp_example)

In [50]:
train_dataloader = DataLoader(train_samples, drop_last=True, batch_size=32)
train_loss = losses.BatchAllTripletLoss(model=model)

In [51]:
num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data #10% 
#logging.info("Warmup-steps: {}".format(warmup_steps))

In [52]:
warmup_steps

153

In [53]:
evaluator = TripletEvaluator.from_input_examples(evaluation_samples)

In [54]:
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          epochs=num_epochs, 
          warmup_steps=warmup_steps, 
          evaluator=evaluator, 
          evaluation_steps=5000, 
          output_path=data_path_models)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/766 [00:00<?, ?it/s]

Iteration:   0%|          | 0/766 [00:00<?, ?it/s]

In [55]:
model = SentenceTransformer(data_path_models)
test_evaluator = TripletEvaluator.from_input_examples(test_samples)
test_evaluator(model, output_path=data_path_models)

0.8924663957486715